In [9]:
from source import seasonalregressor
from source import data_wrangle
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sb
import pickle
from sklearn.metrics import mean_squared_error
%matplotlib inline

advanced = data_wrangle.read_all_advanced('./data/advanced')
pergame  = data_wrangle.read_all_pergame('./data/per_game')
seasonal = data_wrangle.clean_and_join_seasonal_dataframe(advanced, pergame)
demographic = data_wrangle.read_demographic_data('./data/player_data.csv')
fullstats = data_wrangle.add_years_in_league(seasonal, demographic)

In [3]:
cols_to_train_from_correlation = ['WS',
 'OWS',
 'VORP',
 'DWS',
 'MP_total',
 'PS/G',
 'FG',
 'MP_pergame',
 'GS',
 'FT',
 '2P',
 'FTA',
 'FGA',
 '2PA',
 'DRB']

In [4]:
with open('train_test.pkl', 'rb') as f:  # Python 3: open(..., 'rb')
    trainnames, testnames = pickle.load(f)

In [5]:
sr2 = seasonalregressor.SeasonalRegressor(columns_to_train=cols_to_train_from_correlation)

In [6]:
fulltrain, fulltest = sr2.create_train_test_split(fullstats, trainnames, testnames)

In [7]:
sr2 = sr2.fit(fulltrain)

Number of players:  275  with full season data for seasons: {1, 2, 3, 4, 5}
Indices of train set and to-predict set MATCH
Using columns:  ['WS', 'OWS', 'VORP', 'DWS', 'MP_total', 'PS/G', 'FG', 'MP_pergame', 'GS', 'FT', '2P', 'FTA', 'FGA', '2PA', 'DRB']
Fitting for year: 5
Number of players:  275  with full season data for seasons: {1, 2, 3, 4, 6}
Indices of train set and to-predict set MATCH
Using columns:  ['WS', 'OWS', 'VORP', 'DWS', 'MP_total', 'PS/G', 'FG', 'MP_pergame', 'GS', 'FT', '2P', 'FTA', 'FGA', '2PA', 'DRB']
Fitting for year: 6
Number of players:  275  with full season data for seasons: {1, 2, 3, 4, 7}
Indices of train set and to-predict set MATCH
Using columns:  ['WS', 'OWS', 'VORP', 'DWS', 'MP_total', 'PS/G', 'FG', 'MP_pergame', 'GS', 'FT', '2P', 'FTA', 'FGA', '2PA', 'DRB']
Fitting for year: 7
Number of players:  275  with full season data for seasons: {1, 2, 3, 4, 8}
Indices of train set and to-predict set MATCH
Using columns:  ['WS', 'OWS', 'VORP', 'DWS', 'MP_total', 'P

In [8]:
preds = sr2.predict(fulltest)

Predicting year: 5 for player:  Raef LaFrentz
Predicting year: 5 for player:  Brook Lopez
Predicting year: 5 for player:  Rasual Butler
Predicting year: 5 for player:  Jason Williams
Predicting year: 5 for player:  Jared Jeffries
Predicting year: 5 for player:  Bruce Bowen
Predicting year: 5 for player:  Andrea Bargnani
Predicting year: 5 for player:  Antawn Jamison
Predicting year: 5 for player:  Kirk Hinrich
Predicting year: 5 for player:  Robin Lopez
Predicting year: 5 for player:  Mike Dunleavy
Predicting year: 5 for player:  Brian Cook
Predicting year: 5 for player:  Cuttino Mobley
Predicting year: 5 for player:  Andris Biedrins
Predicting year: 5 for player:  Tyronn Lue
Predicting year: 5 for player:  Goran Dragic
Predicting year: 5 for player:  Shawn Marion
Predicting year: 5 for player:  Vitaly Potapenko
Predicting year: 5 for player:  Steve Nash
Predicting year: 5 for player:  Jake Voskuhl
Predicting year: 5 for player:  Stephon Marbury
Predicting year: 5 for player:  Spencer 

In [10]:
actuals = data_wrangle.get_actuals_for_years_5_thru_9(fullstats, testnames)

In [12]:
predictions = sr2.unpack_prediction_dictionary(preds)


In [13]:
score = mean_squared_error(actuals, predictions)

In [14]:
score

5.950312935483871

In [16]:
fullstats.groupby('Player').mean().head()

,Age,G,MP_total,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PS/G,Seasons_number
Player,,,,,,,,,,,,,,,,,,,,,
A.C. Green,35.0,75.800000,1881.000000,12.020000,0.4868,0.021600,0.300200,9.420000,18.560000,13.980000,...,2.060000,4.020000,6.060000,0.860000,0.680000,0.200000,0.640000,1.600000,5.78,14.0
A.J. Bramlett,23.0,8.000000,61.000000,-0.400000,0.1900,0.000000,0.000000,21.700000,18.500000,20.100000,...,1.500000,1.300000,2.800000,0.000000,0.100000,0.000000,0.400000,1.600000,1.00,1.0
A.J. Guyton,23.0,26.666667,415.333333,4.366667,0.3240,0.371000,0.068333,1.366667,3.800000,2.533333,...,0.200000,0.500000,0.700000,1.566667,0.333333,0.133333,0.666667,0.533333,3.80,2.0
A.J. Hammons,24.0,22.000000,163.000000,8.400000,0.4720,0.238000,0.476000,5.400000,21.000000,12.800000,...,0.400000,1.300000,1.600000,0.200000,0.000000,0.600000,0.500000,1.000000,2.20,1.0
A.J. Price,25.5,43.500000,654.833333,11.716667,0.4765,0.473667,0.172333,1.883333,9.033333,5.433333,...,0.233333,1.083333,1.333333,2.000000,0.433333,0.033333,0.800000,0.816667,5.35,3.5


In [18]:
np.average(fullstats["PER"], weights=fullstats["Seasons_number"])

13.019147856103423

In [21]:
from source.helper_functions import weighted_mean_one_col_weight as wm


In [33]:
fullstats.groupby('Player').apply(wm, cols_to_train_from_correlation) - fullstats.groupby('Player').mean()

,2P,2P%,2PA,3P,3P%,3PA,3PAr,AST,AST%,Age,...,TOV,TOV%,TRB,TRB%,TS%,USG%,VORP,WS,WSp48,eFG%
Player,,,,,,,,,,,,,,,,,,,,,
A.C. Green,-0.041429,NaN,-0.074286,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,-2.571429e-02,-0.048571,NaN,NaN
A.J. Bramlett,0.000000,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.000000e+00,0.000000,NaN,NaN
A.J. Guyton,-0.250000,NaN,-0.366667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.000000e-01,-0.066667,NaN,NaN
A.J. Hammons,0.000000,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.000000e+00,0.000000,NaN,NaN
A.J. Price,-0.064286,NaN,-0.150000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,4.761905e-03,-0.092857,NaN,NaN
Aaron Brooks,-0.269508,NaN,-0.580164,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,-1.437705e-01,-0.472459,NaN,NaN
Aaron Gordon,0.470000,NaN,0.935000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.300000e-01,0.200000,NaN,NaN
Aaron Gray,-0.114286,NaN,-0.221429,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,-3.214286e-02,-0.114286,NaN,NaN
Aaron Harrison,0.166667,NaN,0.416667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.000000e+00,-0.016667,NaN,NaN


In [28]:
import importlib

In [29]:
importlib.reload(seasonalregressor)

<module 'source.seasonalregressor' from '/Users/kv/workspace/kv-capstone/NBAwinshare/source/seasonalregressor.py'>

In [30]:
importlib.reload(seasonalregressor)
sr3 = seasonalregressor.SeasonalRegressor(columns_to_train=cols_to_train_from_correlation)
fulltrain, fulltest = sr3.create_train_test_split(fullstats, trainnames, testnames)
sr3 = sr3.fit(fulltrain)

Number of players:  275  with full season data for seasons: {1, 2, 3, 4, 5}
Indices of train set and to-predict set MATCH
Using columns:  ['WS', 'OWS', 'VORP', 'DWS', 'MP_total', 'PS/G', 'FG', 'MP_pergame', 'GS', 'FT', '2P', 'FTA', 'FGA', '2PA', 'DRB']
Fitting for year: 5
Number of players:  275  with full season data for seasons: {1, 2, 3, 4, 6}
Indices of train set and to-predict set MATCH
Using columns:  ['WS', 'OWS', 'VORP', 'DWS', 'MP_total', 'PS/G', 'FG', 'MP_pergame', 'GS', 'FT', '2P', 'FTA', 'FGA', '2PA', 'DRB']
Fitting for year: 6
Number of players:  275  with full season data for seasons: {1, 2, 3, 4, 7}
Indices of train set and to-predict set MATCH
Using columns:  ['WS', 'OWS', 'VORP', 'DWS', 'MP_total', 'PS/G', 'FG', 'MP_pergame', 'GS', 'FT', '2P', 'FTA', 'FGA', '2PA', 'DRB']
Fitting for year: 7
Number of players:  275  with full season data for seasons: {1, 2, 3, 4, 8}
Indices of train set and to-predict set MATCH
Using columns:  ['WS', 'OWS', 'VORP', 'DWS', 'MP_total', 'P

In [31]:
preds = sr3.predict(fulltest)
predictions = sr3.unpack_prediction_dictionary(preds)
score = mean_squared_error(actuals, predictions)

Predicting year: 5 for player:  Raef LaFrentz
Predicting year: 5 for player:  Brook Lopez
Predicting year: 5 for player:  Rasual Butler
Predicting year: 5 for player:  Jason Williams
Predicting year: 5 for player:  Jared Jeffries
Predicting year: 5 for player:  Bruce Bowen
Predicting year: 5 for player:  Andrea Bargnani
Predicting year: 5 for player:  Antawn Jamison
Predicting year: 5 for player:  Kirk Hinrich
Predicting year: 5 for player:  Robin Lopez
Predicting year: 5 for player:  Mike Dunleavy
Predicting year: 5 for player:  Brian Cook
Predicting year: 5 for player:  Cuttino Mobley
Predicting year: 5 for player:  Andris Biedrins
Predicting year: 5 for player:  Tyronn Lue
Predicting year: 5 for player:  Goran Dragic
Predicting year: 5 for player:  Shawn Marion
Predicting year: 5 for player:  Vitaly Potapenko
Predicting year: 5 for player:  Steve Nash
Predicting year: 5 for player:  Jake Voskuhl
Predicting year: 5 for player:  Stephon Marbury
Predicting year: 5 for player:  Spencer 

In [32]:
score

6.404482619354839

In [39]:
importlib.reload(helper_functions)
from source.helper_functions import weighted_mean_multi_col_weight as wm2

In [40]:
fullstats.groupby('Player').apply(wm2, cols_to_train_from_correlation)

,WS,OWS,VORP,DWS,MP_total,PS/G,FG,MP_pergame,GS,FT,2P,FTA,FGA,2PA,DRB
Player,,,,,,,,,,,,,,,
A.C. Green,4.082278,1.772134,0.401659,2.310143,1918.659125,5.738499,2.351320,24.270965,51.257164,1.075113,2.351320,1.609201,5.147964,5.072474,4.010596
A.J. Bramlett,-0.200000,-0.200000,-0.200000,0.100000,61.000000,1.000000,0.500000,7.600000,0.000000,0.000000,0.500000,0.000000,2.600000,2.600000,1.300000
A.J. Guyton,0.141860,0.097674,-0.432558,-0.025581,585.069767,5.302326,2.009302,14.513953,6.232558,0.476744,1.011628,0.546512,5.344186,2.900000,0.693023
A.J. Hammons,0.000000,-0.200000,-0.100000,0.200000,163.000000,2.200000,0.800000,7.400000,0.000000,0.400000,0.500000,0.900000,1.900000,1.500000,1.300000
A.J. Price,0.908867,0.274384,-0.045320,0.598030,688.059113,5.355665,1.965025,14.411823,6.477833,0.676847,1.222660,0.915764,5.188670,2.754187,1.103941
Aaron Brooks,1.733553,0.904955,-0.026357,0.820585,1329.946231,8.507143,3.105456,18.847391,14.533474,1.181787,1.967080,1.386479,7.551819,4.460780,1.189721
Aaron Gordon,3.629926,1.780889,1.030222,1.849037,1959.251852,13.053037,4.924444,28.219556,54.299259,2.094667,3.780593,2.968741,10.977778,7.290963,4.700741
Aaron Gray,0.804121,0.131623,-0.017410,0.698823,550.730866,3.033474,1.253070,12.392431,15.514718,0.480656,1.253070,0.895627,2.466442,2.466442,2.495963
Aaron Harrison,-0.063793,-0.229310,-0.082759,0.129310,145.068966,3.479310,1.050000,14.236207,1.396552,0.758621,0.548276,1.170690,4.048276,1.563793,1.308621


In [45]:
importlib.reload(seasonalregressor)
importlib.reload(helper_functions)
sr3 = seasonalregressor.SeasonalRegressor(columns_to_train=cols_to_train_from_correlation)
fulltrain, fulltest = sr3.create_train_test_split(fullstats, trainnames, testnames)
sr3 = sr3.fit(fulltrain)
preds = sr3.predict(fulltest)
predictions = sr3.unpack_prediction_dictionary(preds)
score = mean_squared_error(actuals, predictions)

ImportError: cannot import name 'weighted_mean_multi_col_weight'